In [255]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns   

In [256]:
#%pip install -U category_encoders


For this competition, there are two target variables:

- h1n1_vaccine - Whether respondent received H1N1 flu vaccine.

- seasonal_vaccine - Whether respondent received seasonal flu vaccine.

Both are binary variables: 0 = No; 1 = Yes. Some respondents didn't get either vaccine, others got only one, and some got both. This is formulated as a multilabel (and not multiclass) problem

The features in this dataset

You are provided a dataset with 36 columns. The first column respondent_id is a unique and random identifier. The remaining 35 features are described below.

For all binary variables: 0 = No; 1 = Yes.

- h1n1_concern - Level of concern about the H1N1 flu.
0 = Not at all concerned; 
1 = Not very concerned; 
2 = Somewhat concerned; 
3 = Very concerned.

- h1n1_knowledge - Level of knowledge about H1N1 flu.
0 = No knowledge; 1 = A little knowledge; 2 = A lot of knowledge.

- behavioral_antiviral_meds - Has taken antiviral medications. (binary)

- behavioral_avoidance - Has avoided close contact with others with flu-like symptoms. (binary)

- behavioral_face_mask - Has bought a face mask. (binary)

- behavioral_wash_hands - Has frequently washed hands or used hand sanitizer. (binary)

- behavioral_large_gatherings - Has reduced time at large gatherings. (binary)

- behavioral_outside_home - Has reduced contact with people outside of own household. (binary)

- behavioral_touch_face - Has avoided touching eyes, nose, or mouth. (binary)

- doctor_recc_h1n1 - H1N1 flu vaccine was recommended by doctor. (binary)

- doctor_recc_seasonal - Seasonal flu vaccine was recommended by doctor. (binary)

- chronic_med_condition - Has any of the following chronic medical conditions: asthma or an other lung condition, diabetes, a heart condition, a kidney condition, sickle cell anemia or other anemia, a neurological or neuromuscular condition, a liver condition, or a weakened immune system caused by a chronic illness or by medicines taken for a chronic illness. (binary)

- child_under_6_months - Has regular close contact with a child under the age of six months. (binary)
health_worker - Is a healthcare worker. (binary)
health_insurance - Has health insurance. (binary)
opinion_h1n1_vacc_effective - Respondent's opinion about H1N1 vaccine effectiveness.
1 = Not at all effective; 2 = Not very effective; 3 = Don't know; 4 = Somewhat effective; 5 = Very effective.

- opinion_h1n1_risk - Respondent's opinion about risk of getting sick with H1N1 flu without vaccine.
1 = Very Low; 2 = Somewhat low; 3 = Don't know; 4 = Somewhat high; 5 = Very high.

- opinion_h1n1_sick_from_vacc - Respondent's worry of getting sick from taking H1N1 vaccine.
1 = Not at all worried; 2 = Not very worried; 3 = Don't know; 4 = Somewhat worried; 5 = Very worried.

- opinion_seas_vacc_effective - Respondent's opinion about seasonal flu vaccine effectiveness.
1 = Not at all effective; 2 = Not very effective; 3 = Don't know; 4 = Somewhat effective; 5 = Very effective.

- opinion_seas_risk - Respondent's opinion about risk of getting sick with seasonal flu without vaccine.
1 = Very Low; 2 = Somewhat low; 3 = Don't know; 4 = Somewhat high; 5 = Very high.

- opinion_seas_sick_from_vacc - Respondent's worry of getting sick from taking seasonal flu vaccine.
1 = Not at all worried; 2 = Not very worried; 3 = Don't know; 4 = Somewhat worried; 5 = Very worried.

- age_group - Age group of respondent.

- education - Self-reported education level.

- race - Race of respondent.

- sex - Sex of respondent.

- income_poverty - Household annual income of respondent with respect to 2008 Census poverty thresholds.

- marital_status - Marital status of respondent.

- rent_or_own - Housing situation of respondent.

- employment_status - Employment status of respondent.

- hhs_geo_region - Respondent's residence using a 10-region geographic classification defined by the U.S. Dept. of Health and Human Services. Values are represented as short random character strings.
census_msa - Respondent's residence within metropolitan statistical areas (MSA) as defined by the U.S. Census.

- household_adults - Number of other adults in household, top-coded to 3.

- household_children - Number of children in household, top-coded to 3.

- employment_industry - Type of industry respondent is employed in. Values are represented as short random character strings.

- employment_occupation - Type of occupation of respondent. Values are represented as short random character strings.



In [257]:
submission_format = pd.read_csv('data/submission_format.csv')  
submission_format.head(3)

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.5,0.7
1,26708,0.5,0.7
2,26709,0.5,0.7


submission_format в целом для нас как я понял это бесполезный файл, с ним работать не прийдется. Это файл пример для сдачи как я понял

In [258]:
training_set_features = pd.read_csv('data/training_set_features.csv')  
training_set_labels = pd.read_csv('data/training_set_labels.csv')  
print(len(training_set_features), len(training_set_labels))
len_tr_set = len(training_set_features)

26707 26707


In [259]:
training_set_features.head(3)

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo


In [260]:
training_set_labels.head(3)

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0


In [261]:
training_set_labels.isna().sum()

respondent_id       0
h1n1_vaccine        0
seasonal_vaccine    0
dtype: int64

In [262]:
training_set_features.isna().sum()

respondent_id                      0
h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

явно видно что у нас большое количество фичей с Nan значениеми, так что надо от них избавиться

In [263]:
for i in training_set_features.columns:
    t = training_set_features[i].isna().sum() / len_tr_set
    if t < 0.03: 
        print(f"{i} ---- 1 ---- {training_set_features[i].isna().sum() / len_tr_set:.3f}%")
    elif t < 0.1:
        print(f"{i} ---- 2 ---- {training_set_features[i].isna().sum() / len_tr_set:.3f}%")
    else:
        print(f"{i} ---- 3 ---- {training_set_features[i].isna().sum() / len_tr_set:.3f}% !!!!!!!!!!!!")

respondent_id ---- 1 ---- 0.000%
h1n1_concern ---- 1 ---- 0.003%
h1n1_knowledge ---- 1 ---- 0.004%
behavioral_antiviral_meds ---- 1 ---- 0.003%
behavioral_avoidance ---- 1 ---- 0.008%
behavioral_face_mask ---- 1 ---- 0.001%
behavioral_wash_hands ---- 1 ---- 0.002%
behavioral_large_gatherings ---- 1 ---- 0.003%
behavioral_outside_home ---- 1 ---- 0.003%
behavioral_touch_face ---- 1 ---- 0.005%
doctor_recc_h1n1 ---- 2 ---- 0.081%
doctor_recc_seasonal ---- 2 ---- 0.081%
chronic_med_condition ---- 2 ---- 0.036%
child_under_6_months ---- 2 ---- 0.031%
health_worker ---- 2 ---- 0.030%
health_insurance ---- 3 ---- 0.460% !!!!!!!!!!!!
opinion_h1n1_vacc_effective ---- 1 ---- 0.015%
opinion_h1n1_risk ---- 1 ---- 0.015%
opinion_h1n1_sick_from_vacc ---- 1 ---- 0.015%
opinion_seas_vacc_effective ---- 1 ---- 0.017%
opinion_seas_risk ---- 1 ---- 0.019%
opinion_seas_sick_from_vacc ---- 1 ---- 0.020%
age_group ---- 1 ---- 0.000%
education ---- 2 ---- 0.053%
race ---- 1 ---- 0.000%
sex ---- 1 ---- 0.000

employment_occupation и employment_industry, health_insurance  тк у них +- 50% Nan я удаляю

In [264]:
df = pd.merge(training_set_features, training_set_labels, on='respondent_id', how='inner')
df.head(5)


,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0


In [265]:
df = df.drop(columns=['respondent_id', 'employment_occupation', 'employment_industry', 'health_insurance'])
df.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,seasonal_vaccine
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,0,0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,0,1
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,0,0


In [266]:
df['h1n1_vaccine'].value_counts()

h1n1_vaccine
0    21033
1     5674
Name: count, dtype: int64

In [267]:
df.isna().sum()

h1n1_concern                     92
h1n1_knowledge                  116
behavioral_antiviral_meds        71
behavioral_avoidance            208
behavioral_face_mask             19
behavioral_wash_hands            42
behavioral_large_gatherings      87
behavioral_outside_home          82
behavioral_touch_face           128
doctor_recc_h1n1               2160
doctor_recc_seasonal           2160
chronic_med_condition           971
child_under_6_months            820
health_worker                   804
opinion_h1n1_vacc_effective     391
opinion_h1n1_risk               388
opinion_h1n1_sick_from_vacc     395
opinion_seas_vacc_effective     462
opinion_seas_risk               514
opinion_seas_sick_from_vacc     537
age_group                         0
education                      1407
race                              0
sex                               0
income_poverty                 4423
marital_status                 1408
rent_or_own                    2042
employment_status           

Дата сет более менее очистил, теперь можно приступать к анализу данных

попробую сделать все в одной ячейке

In [ ]:

import numpy as np
import pandas as pd

df = df.copy() 
target_col = 'h1n1_vaccine'  # или 'seasonal_vaccine'

# 1) Порядковые шкалы: оставить числовыми (0..3/0..5), NaN не трогаем
ordinal_cols = [
    'h1n1_concern','h1n1_knowledge',
    'opinion_h1n1_risk','opinion_h1n1_vacc_effective',
    'opinion_seas_risk','opinion_seas_vacc_effective',
    'opinion_h1n1_sick_from_vacc','opinion_seas_sick_from_vacc'
]
for c in ordinal_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# 2) Булевы/индикаторы: приводим к 0/1/NaN
bool_like = [
    'behavioral_antiviral_meds','behavioral_avoidance','behavioral_face_mask',
    'behavioral_wash_hands','behavioral_large_gatherings','behavioral_outside_home',
    'behavioral_touch_face','doctor_recc_h1n1','doctor_recc_seasonal',
    'chronic_med_condition','child_under_6_months','health_worker'
]
for c in bool_like:
    if c in df.columns:
        # если строки 'True'/'False' – сначала map; если уже 0/1/NaN – будет no-op
        df[c] = df[c].map({True:1, False:0}).astype('float')

# 3) Номинальные категориальные: оставить строками (CatBoost сам закодирует)
cat_nominal = [
    'age_group','education','race','sex','income_poverty',
    'marital_status','rent_or_own','employment_status',
    'hhs_geo_region','census_msa'
]
for c in cat_nominal:
    if c in df.columns:
        df[c] = df[c].astype('string')  # допускает NaN

# 4) Цель и признаки
y = df[target_col].astype(int)
X = df.drop(columns=[target_col])


In [269]:
X = X.replace({pd.NA: np.nan})
y = y.replace({pd.NA: np.nan})

# 1.2 Определить категориальные колонки (номинальные, которые надо отдать CatBoost)
# Это колонки с малым кардиналом или явно не числовые:
cat_nominal = [
    'age_group', 'education', 'race', 'sex', 'income_poverty',
    'marital_status', 'rent_or_own', 'employment_status',
    'hhs_geo_region', 'census_msa'
]

# Проверяем, какие есть в X
cat_nominal = [c for c in cat_nominal if c in X.columns]

# Преобразуем их в строки (fillna('missing') или оставить NaN как есть — CatBoost умеет)
for c in cat_nominal:
    # Вариант A: оставить NaN (CatBoost сам обработает)
    X[c] = X[c].astype('str').replace('nan', np.nan)
    # Вариант B: заполнить явно
    # X[c] = X[c].fillna('missing').astype('str')

# 1.3 Убедиться, что ordinal/числовые признаки — numeric
ordinal_cols = [
    'h1n1_concern', 'h1n1_knowledge',
    'opinion_h1n1_risk', 'opinion_h1n1_vacc_effective',
    'opinion_seas_risk', 'opinion_seas_vacc_effective',
    'opinion_h1n1_sick_from_vacc', 'opinion_seas_sick_from_vacc'
]
for c in ordinal_cols:
    if c in X.columns:
        X[c] = pd.to_numeric(X[c], errors='coerce')

# Булевы/индикаторы -> 0/1/NaN
bool_cols = [
    'behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask',
    'behavioral_wash_hands', 'behavioral_large_gatherings', 'behavioral_outside_home',
    'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
    'chronic_med_condition', 'child_under_6_months', 'health_worker'
]
for c in bool_cols:
    if c in X.columns:
        X[c] = X[c].map({True:1, False:0, 1:1, 0:0}).astype('float')

# 1.4 Целевая переменная — int без NaN
y = y.astype(int)

# 1.5 Индексы категориальных для CatBoost
cat_features = [X.columns.get_loc(c) for c in cat_nominal]

In [270]:
# 2.1 Индикаторы пропусков для ключевых ordinal шкал
for c in ordinal_cols:
    if c in X.columns:
        X[f'{c}_is_na'] = X[c].isna().astype('float')

# 2.2 Агрегаты по тематике (concern/risk/effective)
concern_cols = ['h1n1_concern', 'opinion_h1n1_risk', 'opinion_seas_risk']
concern_cols_exist = [c for c in concern_cols if c in X.columns]
if concern_cols_exist:
    X['concern_risk_sum'] = X[concern_cols_exist].sum(axis=1, skipna=True)
    X['concern_risk_mean'] = X[concern_cols_exist].mean(axis=1, skipna=True)

effective_cols = ['opinion_h1n1_vacc_effective', 'opinion_seas_vacc_effective']
effective_cols_exist = [c for c in effective_cols if c in X.columns]
if effective_cols_exist:
    X['vacc_effective_sum'] = X[effective_cols_exist].sum(axis=1, skipna=True)

# 2.3 Индикатор "обе рекомендации врача"
if 'doctor_recc_h1n1' in X.columns and 'doctor_recc_seasonal' in X.columns:
    X['doctor_recc_both'] = ((X['doctor_recc_h1n1']==1) & (X['doctor_recc_seasonal']==1)).astype('float')

# 2.4 Взаимодействие "знание × беспокойство"
if 'h1n1_knowledge' in X.columns and 'h1n1_concern' in X.columns:
    X['knowledge_x_concern'] = X['h1n1_knowledge'] * X['h1n1_concern']

# 2.5 Число пропусков в ключевых opinion-колонках
opinion_cols = [c for c in X.columns if 'opinion' in c]
X['opinion_na_count'] = X[opinion_cols].isna().sum(axis=1)


In [271]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve

# Индексы категориальных колонок для CatBoost
cat_features = [X.columns.get_loc(c) for c in cat_nominal if c in X.columns]

# Стратифицированная K-fold валидация
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=314)


In [272]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve
import numpy as np

oof_proba = np.zeros(len(X))
models = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=314)

for fold, (tr, va) in enumerate(skf.split(X, y), 1):
    X_tr, X_va = X.iloc[tr].copy(), X.iloc[va].copy()
    y_tr, y_va = y.iloc[tr], y.iloc[va]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
    valid_pool = Pool(X_va, y_va, cat_features=cat_features)

    model = CatBoostClassifier(
        iterations=4000,
        depth=5,                    
        learning_rate=0.03,
        loss_function='Logloss',
        eval_metric='AUC',
        l2_leaf_reg=5.0,            
        random_strength=1.0,
        border_count=254,
        auto_class_weights='Balanced',
        subsample=0.8,              
        random_state=314,
        task_type='CPU',
        early_stopping_rounds=300,
        verbose=200
    )
    
    model.fit(train_pool, eval_set=valid_pool)
    models.append(model)

    oof_proba[va] = model.predict_proba(valid_pool)[:, 1]
    print(f"Fold {fold} done\n")

# Метрики
auc = roc_auc_score(y, oof_proba)
ths = np.linspace(0.05, 0.95, 181)
f1s = [f1_score(y, (oof_proba > t).astype(int)) for t in ths]
best_t = float(ths[int(np.argmax(f1s))])
best_f1 = float(np.max(f1s))
print(f"AUC_OOF: {auc:.4f} | Best threshold: {best_t:.2f} | F1_OOF: {best_f1:.4f}")

prec, rec, thr = precision_recall_curve(y, oof_proba)
f1s_pr = 2*prec*rec/(prec+rec+1e-12)
print(f"F1 via PR best: {np.nanmax(f1s_pr):.4f}")



0:	test: 0.7991567	best: 0.7991567 (0)	total: 85ms	remaining: 5m 39s
200:	test: 0.8376772	best: 0.8377120 (196)	total: 19.7s	remaining: 6m 12s
400:	test: 0.8401350	best: 0.8401392 (399)	total: 38.4s	remaining: 5m 44s
600:	test: 0.8405124	best: 0.8408272 (495)	total: 57s	remaining: 5m 22s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8408271718
bestIteration = 495

Shrink model to first 496 iterations.
Fold 1 done

0:	test: 0.8008533	best: 0.8008533 (0)	total: 79.3ms	remaining: 5m 17s
200:	test: 0.8428867	best: 0.8428867 (200)	total: 19.8s	remaining: 6m 13s
400:	test: 0.8464731	best: 0.8464731 (400)	total: 40.2s	remaining: 6m 1s
600:	test: 0.8473835	best: 0.8474061 (597)	total: 1m 1s	remaining: 5m 47s
800:	test: 0.8481051	best: 0.8481103 (789)	total: 1m 22s	remaining: 5m 30s
1000:	test: 0.8479208	best: 0.8482077 (812)	total: 1m 42s	remaining: 5m 7s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8482076757
bestIteration = 812

Shrink model to 

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import numpy as np
import pandas as pd


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=314)


base_model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    random_state=314,
    task_type='GPU',
    verbose=False,              
    cat_features=cat_features,
    early_stopping_rounds=200   # ранняя остановка для каждого кандидата
)


param_grid = {
    'iterations': [2000],       
    'depth': [4, 5, 6],
    'learning_rate': [0.02, 0.03, 0.05],
    'l2_leaf_reg': [3, 5, 7],
    'subsample': [0.7, 0.8],
    'random_strength': [0.5, 1.0],
    'border_count': [128, 254]
}


grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    cv=cv,
    scoring='roc_auc',          
    n_jobs=1,                   
    verbose=2,
    return_train_score=False
)

print("Starting GridSearchCV...")
grid_search.fit(X, y)

# Лучшие параметры и скор
print("\n" + "="*70)
print("BEST PARAMETERS:")
print(grid_search.best_params_)
print(f"\nBest CV AUC: {grid_search.best_score_:.4f}")
print("="*70)

# Лучшая модель
best_model = grid_search.best_estimator_


Starting GridSearchCV...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END border_count=128, depth=4, iterations=2000, l2_leaf_reg=3, learning_rate=0.02, random_strength=0.5, subsample=0.7; total time= 2.2min
[CV] END border_count=128, depth=4, iterations=2000, l2_leaf_reg=3, learning_rate=0.02, random_strength=0.5, subsample=0.7; total time= 2.2min
[CV] END border_count=128, depth=4, iterations=2000, l2_leaf_reg=3, learning_rate=0.02, random_strength=0.5, subsample=0.7; total time= 2.2min
[CV] END border_count=128, depth=4, iterations=2000, l2_leaf_reg=3, learning_rate=0.02, random_strength=0.5, subsample=0.7; total time= 2.2min
[CV] END border_count=128, depth=4, iterations=2000, l2_leaf_reg=3, learning_rate=0.02, random_strength=0.5, subsample=0.7; total time= 2.2min
[CV] END border_count=128, depth=4, iterations=2000, l2_leaf_reg=3, learning_rate=0.02, random_strength=0.5, subsample=0.8; total time= 2.2min
[CV] END border_count=128, depth=4, iterations=2000, 